In [ ]:
# Transformer 모델 구축 - Transformer News Summary 요약 모델
# 학습 목표 - 실무에서 사용되는 파이프라인 이해 및 적용
# - 1. 데이터 로드 & 확인: 결측치 제거(None, "")

In [ ]:
# 데이터셋 로드 및 json 파일 추출
import os
import json

data_dir = './llm_data/ai_hub_news_summary' # 데이터셋 경로

all_data = []

for file_name in os.listdir(data_dir): # 파일명 추출
    if file_name.endswith('.json'): # .json 파일 추출
        with open(os.path.join(data_dir, file_name), 'r', encoding='utf-8') as f: # 파일 일기
            data = json.load(f) # json 파일 로드
            all_data.append(data)

In [ ]:
# JSON 파싱: 본문과 요약 추출
dataset = [] # 최종적으로 본문과 요약문을 담는 리스트

for item in all_data: # JSON 데이터셋    
    passage = item['Meta(Refine)']['passage'] # 본문

    summaries = [] # 요약문
    if item['Annotation'].get('summary1'):
        summaries.append(item['Annotation'].get('summary1'))
    if item['Annotation'].get('summary2'):        
        summaries.append(item['Annotation'].get('summary2'))

    # 요약이 없는 경우 제외
    if passage and summaries:
        dataset.append({'text': passage, 'summary': summaries})

In [ ]:
# 데이터 전처리
def clean_text(text):
    # 줄바꿈 -> 공백으로 바꾼다, 문장열 앞뒤 불필요한 공백 제거
    text = text.replace('\n', ' ').strip()
    return text

for sample in dataset:
    sample['text'] = clean_text(sample['text']) # 문자열
    sample['summary'] = [ clean_text(s) for s in sample['summary'] ] # 리스트

# 데이터 확인
print(len(dataset), dataset[0])

In [ ]:
# 토크나이저 적용 데이터셋 -> DataLoader 생성, collate_fn 적용
import torch
from transformers import BartTokenizer
from torch.utils.data import DataLoader, random_split

# BART 토크라이저 로드, Bart vocab size: 50265
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
# print('Bart vocab size:', tokenizer.vocab_size)

# collate_fn 정의
def collate_fn(batch):
    texts = [ item['text'] for item in batch ]
    summaries = [ ' '.join(item['summary']) for item in batch ]

    # 입력과 요약을 토큰화
    inputs = tokenizer(
        texts, # 본문 리스트
        max_length=1024, # BART 입력 최대 길이
        truncation=True,
        padding='max_length', # padding='max_length' max_length까지 모든 문장을 강제 패딩, padding=True 배치 내에서 가장 긴 문장 길이에 맞추어 패딩
        return_tensors='pt'
    )
    labels = tokenizer(
        summaries, # 요약 리스트를 하나의 문자열로 합치기
        max_length=128, # 요약 최대 길이
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': labels['input_ids']        
    }
dataset_small = dataset[:1000] # 데이터 축소
total_size = len(dataset_small) # 전체 데이터 길이
train_size = int(0.8 * total_size) # 비율 설정(예시: train 80%, val 10%, test 10%)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size
generator = torch.Generator().manual_seed(42) # 시드 고정
train_dataset, val_dataset, test_dataset = random_split(dataset_small, [train_size, val_size, test_size], generator=generator) # 데이터 분리

# DataLoader 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=collate_fn
)
val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn
)

# 배치 확인
batch = next(iter(train_loader))
print(batch['input_ids'].shape, batch['labels'].shape)

In [ ]:
# 모델 정의
# - Feature Extraction + LoRA Fine-tuning 조합

import torch
from transformers import BartForConditionalGeneration
from peft import LoraConfig, get_peft_model
from torch.amp import autocast, GradScaler
import os

# GPU 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'PyTorch Version: {torch.__version__}, Device: {device}')

# BartForConditionalGeneration 베이스 모델
model = BartForConditionalGeneration.from_pretrained(model_name)

# Feature Extraction
for name, param in model.named_parameters():
    if 'lora' not in name: # LoRA 모듈이 아닌 경우
        param.requires_grad = False # 모델 본체 동결

# LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias='none',
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, lora_config)
model.to(device)

# 모델 확인, trainable params: 2,359,296 || all params: 408,649,728 || trainable%: 0.5773
model.print_trainable_parameters() # LoRA 적용 확인
print(model)

# 최적화 설정
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scaler = GradScaler()

# 반복횟수
num_epochs = 3

# Early Stopping 클래스 정의
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.0, path='./llm_models/19_transformer_summary_news/best_model.pt'):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
        self.path = path
    
    def __call__(self, valid_loss, model):
        if self.best_loss is None:
            self.best_loss = valid_loss
            self.save_checkpoint(model)
        # 성능 개선 -> 최적 모델 갱신
        elif valid_loss < self.best_loss - self.min_delta:
            self.best_loss = valid_loss
            self.counter = 0
            self.save_checkpoint(model)
        # 개선 없음
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
    
    def save_checkpoint(self, model):
        # 디렉토리만 생성
        folder = os.path.dirname(self.path)
        if folder !='' and not os.path.exists(folder):
            os.makedirs(folder)
        
        torch.save(model.state_dict(), self.path)
        print(f'Best model saved at {self.path}')
early_stopping = EarlyStopping(patience=3, min_delta=0.001)

In [ ]:
# 학습 루프: autocast(속도 향상) 적용, GradScaler(안정적 학습) 적용
# autocast 적용: 연산을 FP16(half precision)과 FP32(full precision)중 적절히 선택해서 실행
# - 속도 향상: 대부분의 연산을 FP16으로 처리해 GPU 연산 속도를 높인다
# - 안정성 유지: 손실이 큰 연산(예시:소프트맥스,레이어정규화)은 FP32로 자동 변환해 정확도를 보장한다
# GradScaler 적용: FP16 학습에서는 작은 값이 underflow(0으로 사라짐)될 위험이 있다
# - 안정적 학습 보장: GradScaler는 손실(loss)를 크게 스케일링해서 역전파 시 그래디언트가 사라지지 않도록 한다
# - 이후 업데이트 단계에서 다시 원래 크기로 되돌려 안정적인 학습을 보장한다. 즉 FP16 학습에서 발생할 수 있는 수치 불안정 문제를 해결하는 역할

from tqdm import tqdm

for epoch in range(num_epochs):
    # train
    model.train() # 학습 모드 지정
    total_loss = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1} [Train]'):
        optimizer.zero_grad() # 오차역전파 코드, 미분 전 가중치/바이어스 파라미터 초기화

        # 학습데이터 GPU 지정
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # AMP(Automatic Mixed Precision) GPU에서 연산 속도와 메모리 효율 향상
        with autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels) # 모델 예측
            loss = outputs.loss # 손실값
            logits = outputs.logits # Softmax 계산된 logits값
        
        # 오차역전파(GradScaler로 안정성 확보)
        scaler.scale(loss).backward() # 미분연산
        scaler.step(optimizer) # 미분 연산 후 가중치/바이어스 파라미터 업데이트
        scaler.update()

        total_loss += loss.item() # 손실 누적
    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f}')

    # validation
    model.eval() # 검증/추론 모드 지정
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f'Epoch {epoch+1} [Validation]'):
            # 검증데이터 GPU 지정
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # AMP(Automatic Mixed Precision) GPU에서 연산 속도와 메모리 효율 향상
            with autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels) # 모델 예측
                loss = outputs.loss # 손실값
                # logits = outputs.logits # Softmax 계산된 logits값
            
                val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    # Early Stopping 객체 호출
    early_stopping(val_loss, model)
    status = 'STOP' if early_stopping.early_stop else 'CONTINUE' # # Early Stopping status 상태값

    print(f'Epoch {epoch+1} | Validation Loss: {avg_val_loss:.4f}')
    
    # Early Stopping 체크
    if early_stopping.early_stop: # early_stop=True 학습 종료
        print('Early stopping triggered')
        break 

In [ ]:
# # 최적 모델 로드

# torch.load() 파일에서 파라미터(가중치) 딕셔너리를 불러옴
# model.load_state_dict() 불러온 파리미터를 모델 구조에 맞게 적용
model.load_state_dict(torch.load('./llm_models/19_transformer_summary_news/best_model.pt'))

# 모델을 실행할 디바이스(GPU or CPU)에 올린다
model.to(device)

# 검증/추론 모드 전환 
# - model.eval() 검증/추촌 모드 에서는 Dropout 등이 비활성화되어 일관된 추론 결과를 보장한다, 
# - model.train() 학습 모드 에서는 Dropout, BatchNorm 등이 활성화되어 파라미터 업데이트를 준비 한다
model.eval() 

In [ ]:
# 테스트
model.eval() # 검증/추론 모드 지정
test_loss = 0

with torch.no_grad(): # 미분 연산 하지 않음
    for batch in tqdm(test_loader, desc=f'Epoch {epoch+1} [Test]'): # 테스트 데이터 GPU 지정
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # AMP(Automatic Mixed Precision) GPU에서 연산 속도와 메모리 효율 향상
        with autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss        
        test_loss += loss.item() # 손실값 누적    
avg_test_loss = test_loss / len(test_loader) # 평균 손실값 게산
print(f'Final Test Loss: {avg_test_loss:.4f}')

In [ ]:
# 실제 요약 생성
sample = test_dataset[0] # 테스트 데이터셋에서 첫번째 샘플
text = sample['text'] # 기사 본문 추출

# 본문을 토크나이저로 인코딩 -> GPU로 이동
inputs = tokenizer( 
    text, # 본문(text)을 BART 모델이 이해할 수 있는 토큰ID(input_ids)와 어텐션 마스크(attention_mask)로 변환
    max_length=1024, # 입력 문장을 최대 1024 토큰까지 자름
    truncation=True,
    padding='max_length', # 모든 문장을 동일 길이(1024)로 맞추기 위해 패딩 추가
    return_tensors='pt'
).to(device)

# 학습된 모델로 요약 생성
summary_ids = model.generate(
    inputs=inputs['input_ids'], # input_ids, attention_mask를 모델에 넣어 요약 생성
    attention_mask=inputs['attention_mask'],
    max_length=128, # 요약 최대 길이
    num_beams=4, # Beam Search로 다양한 후보 탐색
    early_stopping=True # EOS 토큰 나오면 조기 종료
)

# 결과 디코딩, 모델이 생성한 토큰 ID를 사람이 읽을 수 있는 문자열로 변환, <pad>/<eos> 같은 특수 토큰 제거
print('Generated Summary:', tokenizer.decode(summary_ids[0], skip_special_tokens=True))
print('Reference Summary:', ' '.join(sample['summary']))